In [1]:
# Imports

import optuna
import mlflow
import dagshub
import mlflow.sklearn

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from imblearn.under_sampling import RandomUnderSampler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [2]:
# Setting up DagsHub

dagshub.init(repo_owner='SushrutGaikwad', repo_name='youtube-comments-analyzer', mlflow=True)

Accessing as SushrutGaikwad

Initialized MLflow to track repo "SushrutGaikwad/youtube-comments-analyzer"

Repository SushrutGaikwad/youtube-comments-analyzer initialized!

# Data

In [3]:
PREPROCESSED_DATA_PATH = "../data/processed/reddit_preprocessed.csv"
df = pd.read_csv(PREPROCESSED_DATA_PATH)
df.dropna(subset=["clean_comment"], inplace=True)
df.shape

(36662, 2)

# Running the experiment

In [4]:
# Setting experiment name

mlflow.set_experiment("Exp 4: ML algorithms with hyperparameter tuning")

<Experiment: artifact_location='mlflow-artifacts:/3f66177ebab24d6392ce8f143c28100b', creation_time=1749046747756, experiment_id='5', last_update_time=1749046747756, lifecycle_stage='active', name='Exp 4: ML algorithms with hyperparameter tuning', tags={}>

## Preprocessing

In [5]:
# Remapping class labels from {-1, 0, 1} to {2, 0, 1}
mapping = {
    -1: 2,
    0: 0,
    1: 1
}
df["category"] = df["category"].map(mapping)

# Removing missing sentiments
df.dropna(subset=["category"], inplace=True)

# Feature engineering
ngram_range = (1, 2)
max_features = 1000
vectorizer = CountVectorizer(
    ngram_range=ngram_range,
    max_features=max_features
)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    df["clean_comment"],
    df["category"],
    test_size=0.2,
    random_state=42,
    stratify=df["category"]
)
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Undersampling
rus = RandomUnderSampler(random_state=42)
X_train_vectorized, y_train = rus.fit_resample(
    X_train_vectorized,
    y_train
)

## Running the experiment

In [6]:
# Function to log results to MLFlow
def log_to_mlflow(
    model_name,
    model,
    X_train,
    X_test,
    y_train,
    y_test
):
    with mlflow.start_run():
        # Tags
        mlflow.set_tag(
            "mlflow.runName", f"{model_name}_BoW_bigrams_1000_undersampling"
        )
        mlflow.set_tag("experiment_type", "ML_models_comparison")
        
        # Logging model name as a parameter
        mlflow.log_param("model_name", model_name)
        
        # Initializing and training the model
        model.fit(X_train, y_train)
        
        # Making predictions on the test set and logging metrics
        y_pred = model.predict(X_test)
        
        # Logging accuracy
        accuracy = accuracy_score(
            y_true=y_test,
            y_pred=y_pred
        )
        mlflow.log_metric("accuracy", accuracy)
        
        # Logging classification report metrics
        classification_rep = classification_report(
            y_true=y_test,
            y_pred=y_pred,
            output_dict=True
        )
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}: {metric}", value)
        
        # Logging the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")

In [7]:
# Optuna objective function
def objective(trial):
    n_neighbors = trial.suggest_int("n_neighbors", 3, 30)
    p = trial.suggest_categorical(
        "p", [1, 2]
    )
    
    model = KNeighborsClassifier(
        n_neighbors=n_neighbors, p=p, n_jobs=-1
    )
    model.fit(X_train_vectorized, y_train)
    y_pred = model.predict(X_test_vectorized)
    return accuracy_score(
        y_true=y_test,
        y_pred=y_pred
    )

In [8]:
def run_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=50)
    
    # Getting the best parameters and logging the best model
    best_params = study.best_params
    best_model = KNeighborsClassifier(
        n_neighbors=best_params["n_neighbors"],
        p=best_params["p"],
        n_jobs=-1
    )
    log_to_mlflow(
        model_name="KNN",
        model=best_model,
        X_train=X_train_vectorized,
        X_test=X_test_vectorized,
        y_train=y_train,
        y_test=y_test
    )

In [9]:
run_experiment()

[I 2025-06-04 21:21:06,053] A new study created in memory with name: no-name-25d2d4cc-d42d-4cf9-bf58-6bb119fa0c32
[I 2025-06-04 21:21:08,771] Trial 0 finished with value: 0.47347606709395884 and parameters: {'n_neighbors': 13, 'p': 2}. Best is trial 0 with value: 0.47347606709395884.
[I 2025-06-04 21:21:11,169] Trial 1 finished with value: 0.4936588026728488 and parameters: {'n_neighbors': 4, 'p': 2}. Best is trial 1 with value: 0.4936588026728488.
[I 2025-06-04 21:21:13,578] Trial 2 finished with value: 0.4520660030001364 and parameters: {'n_neighbors': 19, 'p': 2}. Best is trial 1 with value: 0.4936588026728488.
[I 2025-06-04 21:21:19,383] Trial 3 finished with value: 0.46720305468430384 and parameters: {'n_neighbors': 6, 'p': 1}. Best is trial 1 with value: 0.4936588026728488.
[I 2025-06-04 21:21:22,191] Trial 4 finished with value: 0.46720305468430384 and parameters: {'n_neighbors': 6, 'p': 1}. Best is trial 1 with value: 0.4936588026728488.
[I 2025-06-04 21:21:24,423] Trial 5 fini

🏃 View run KNN_BoW_bigrams_1000_undersampling at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/5/runs/54821890f7c64da5861a073248be71d6
🧪 View experiment at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/5
